In [72]:
import os
from dataclasses import dataclass
import torch
import numpy as np
import math
from tqdm import tqdm
import json 
import pandas as pd

from src.tokenizers.eng import EnglishTokenizer
from src.tokenizers.indic import IndicTokenizer
from src.components import TransformerMT

In [73]:
def translate_batch(model, eng_tokenizer, indic_tokenizer, english_texts, device='cuda', verbose=False):
    
    # all texts to sequences at once
    english_ids = eng_tokenizer.texts_to_sequences(english_texts)
    english_tensor = torch.tensor(english_ids, device=device)
    
    with torch.no_grad():
        # generate translations for the entire batch from model.generate
        translation_ids = model.generate(english_tensor, max_length=20, temperature=0.0)
    
    if verbose:
        print(f"Batch size: {len(english_texts)}")
        print(f"Generated shape: {translation_ids.shape}")

    # decode from indic_detokenizer
    translation_array = translation_ids.cpu().numpy()
    indic_texts = indic_tokenizer.sequences_to_texts(translation_array)
    
    return indic_texts

In [74]:
test_data = json.load(open(os.path.join("data", "raw", "val_data1.json")))

### ENG 2 HINDI

In [75]:
@dataclass
class TransformerConfig:
    SRC_VOCAB_SIZE: int = 30_000                      # source vocabulary size
    TGT_VOCAB_SIZE: int = 30_000                      # target vocabulary size
    SRC_MAX_LENGTH: int = 256                         # max sequence length source lang
    TGT_MAX_LENGTH: int = 256                         # max sequence length target lang
    D_MODEL: int = 128                                # embedding dimension
    N_HEADS: int = 4                                  # number of heads in attention
    N_LAYERS: int = 6                                 # number of transformer blocks
    D_FF: int = 128 * 4                               # dimension of feedforward (4x of embedding dims)
    MAX_SEQ_LEN: int = 256
    DROPOUT: float = 0.1
    BATCH_SIZE: int = 32
    EVAL_STEPS: int = 100
    EPOCHS: int = 10

In [77]:
hindi_checkpoint_path = os.path.join("checkpoints", "eng_hindi", "exp3-eng-hindi-transformer")
checkpoint = torch.load(os.path.join(hindi_checkpoint_path, "tx_epoch_10_step_22500.pt"), weights_only=False)

In [78]:
# recreate ENGLISH tokenizer
eng_tok = EnglishTokenizer(checkpoint['eng_tokenizer']['max_vocab_size'], 
                           checkpoint['eng_tokenizer']['max_length'])
eng_tok.word2idx = checkpoint['eng_tokenizer']['word2idx']
eng_tok.idx2word = checkpoint['eng_tokenizer']['idx2word']
eng_tok.vocab_size = checkpoint['eng_tokenizer']['vocab_size']

In [79]:
# recreate INDIC tokenizer
indic_tok = IndicTokenizer(checkpoint['indic_tokenizer']['max_vocab_size'], 
                           checkpoint['indic_tokenizer']['max_length'])
indic_tok.word2idx = checkpoint['indic_tokenizer']['word2idx']
indic_tok.idx2word = checkpoint['indic_tokenizer']['idx2word']
indic_tok.vocab_size = checkpoint['indic_tokenizer']['vocab_size']

In [80]:
model_config = checkpoint['model_config']
model = TransformerMT(**model_config)
model.to('cuda')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [81]:
translation = translate_batch(model, eng_tok, indic_tok, english_texts=["Hi"] , device='cuda')

In [82]:
hindi_output = pd.DataFrame()
ids = []
texts = []
for id_, entry in test_data['English-Hindi']['Validation'].items():
    ids.append(id_)
    texts.append(entry['source'])

print(f"Total samples: {len(texts)}")
print(f"Processing in batches of {32}")

all_translations = []
for i in tqdm(range(0, len(texts), 32), desc="Processing batches"):
    batch_texts = texts[i:i+32]
    batch_translations = translate_batch(model, eng_tok, indic_tok, batch_texts, device='cuda')
    all_translations.extend(batch_translations)

# gather in df
hindi_output['ID'] = ids
hindi_output['Translation'] = all_translations

Total samples: 11543
Processing in batches of 32


Processing batches: 100%|████████████████████████████████████████████████████████████████████████████| 361/361 [00:29<00:00, 12.24it/s]


### ENG 2 BENGALI

In [83]:
@dataclass
class TransformerConfig:
    SRC_VOCAB_SIZE: int = 30_000                      # source vocabulary size
    TGT_VOCAB_SIZE: int = 30_000                      # target vocabulary size
    SRC_MAX_LENGTH: int = 256                         # max sequence length source lang
    TGT_MAX_LENGTH: int = 256                         # max sequence length target lang
    D_MODEL: int = 128                                # embedding dimension
    N_HEADS: int = 4                                  # number of heads in attention
    N_LAYERS: int = 6                                 # number of transformer blocks
    D_FF: int = 128 * 4                               # dimension of feedforward (4x of embedding dims)
    MAX_SEQ_LEN: int = 256
    DROPOUT: float = 0.1
    BATCH_SIZE: int = 32
    EVAL_STEPS: int = 100
    EPOCHS: int = 10


In [84]:
bengali_checkpoint_path = os.path.join("checkpoints", "eng_bengali", "exp3-eng-bengali-transformer")
checkpoint = torch.load(os.path.join(bengali_checkpoint_path, "tx_epoch_10_step_19000.pt"), weights_only=False)

In [85]:
# recreate ENGLISH tokenizer
eng_tok = EnglishTokenizer(checkpoint['eng_tokenizer']['max_vocab_size'], 
                           checkpoint['eng_tokenizer']['max_length'])
eng_tok.word2idx = checkpoint['eng_tokenizer']['word2idx']
eng_tok.idx2word = checkpoint['eng_tokenizer']['idx2word']
eng_tok.vocab_size = checkpoint['eng_tokenizer']['vocab_size']

In [86]:
# recreate INDIC tokenizer
indic_tok = IndicTokenizer(checkpoint['indic_tokenizer']['max_vocab_size'], 
                           checkpoint['indic_tokenizer']['max_length'])
indic_tok.word2idx = checkpoint['indic_tokenizer']['word2idx']
indic_tok.idx2word = checkpoint['indic_tokenizer']['idx2word']
indic_tok.vocab_size = checkpoint['indic_tokenizer']['vocab_size']

In [87]:
model_config = checkpoint['model_config']
model = TransformerMT(**model_config)
model.to('cuda')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [88]:
translation = translate_batch(model, eng_tok, indic_tok, english_texts=["Hi"] , device='cuda')

In [89]:
bengali_output = pd.DataFrame()
ids = []
texts = []
for id_, entry in test_data['English-Bengali']['Validation'].items():
    ids.append(id_)
    texts.append(entry['source'])

print(f"Total samples: {len(texts)}")
print(f"Processing in batches of {32}")

all_translations = []
for i in tqdm(range(0, len(texts), 32), desc="Processing batches"):
    batch_texts = texts[i:i+32]
    batch_translations = translate_batch(
        model, eng_tok, indic_tok, batch_texts, device='cuda'
    )
    all_translations.extend(batch_translations)


bengali_output['ID'] = ids
bengali_output['Translation'] = all_translations

Total samples: 9836
Processing in batches of 32


Processing batches: 100%|████████████████████████████████████████████████████████████████████████████| 308/308 [00:24<00:00, 12.37it/s]


## Final output for submission

In [90]:
output = pd.concat([bengali_output, hindi_output]).reset_index(drop=True)

In [91]:
# output['Translation'] = output['Translation'].str.replace(",", "")

In [92]:
output.to_csv(
    "answers/val/answer3.csv",
    sep="\t",
    index=False,
    header=True,
    quoting=csv.QUOTE_ALL,
    lineterminator="\n",
    doublequote=True,
)

In [36]:
# output.to_csv("answers/val/answer1.csv", index=False)

In [93]:
# answer = "answers/val/answer1.csv"
# with open(answer, "w", newline="", encoding="utf-8") as f:
#     writer = csv.writer(f, delimiter="\t", quoting=csv.QUOTE_MINIMAL)
#     writer.writerow(["ID", "Translation"])  # header
#     for i in range(output.shape[0]):
#         writer.writerow([output["ID"][i], output["Translation"][i]])